In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
cd  '/content/drive/MyDrive/Seg3D/KiTS2019/kits19'

/content/drive/MyDrive/Seg3D/KiTS2019/kits19


In [5]:
import pandas as pd 
import numpy as np
import torch
import torch.nn as nn 
import nibabel as nib
from pathlib import Path
from segment.utils.utils import multiprocess

In [7]:
data = pd.read_csv("/content/drive/MyDrive/Seg3D/KiTS2019/kits19/data/train_ds.csv", index_col=0)
data

,case_id,img_path,seg_path,mask
0,case_00000,/content/drive/MyDrive/Seg3D/KiTS2019/kits19/d...,/content/drive/MyDrive/Seg3D/KiTS2019/kits19/d...,1
1,case_00001,/content/drive/MyDrive/Seg3D/KiTS2019/kits19/d...,/content/drive/MyDrive/Seg3D/KiTS2019/kits19/d...,1
2,case_00002,/content/drive/MyDrive/Seg3D/KiTS2019/kits19/d...,/content/drive/MyDrive/Seg3D/KiTS2019/kits19/d...,1
3,case_00003,/content/drive/MyDrive/Seg3D/KiTS2019/kits19/d...,/content/drive/MyDrive/Seg3D/KiTS2019/kits19/d...,1
4,case_00004,/content/drive/MyDrive/Seg3D/KiTS2019/kits19/d...,/content/drive/MyDrive/Seg3D/KiTS2019/kits19/d...,1
...,...,...,...,...
205,case_00205,/content/drive/MyDrive/Seg3D/KiTS2019/kits19/d...,/content/drive/MyDrive/Seg3D/KiTS2019/kits19/d...,1
206,case_00206,/content/drive/MyDrive/Seg3D/KiTS2019/kits19/d...,/content/drive/MyDrive/Seg3D/KiTS2019/kits19/d...,1
207,case_00207,/content/drive/MyDrive/Seg3D/KiTS2019/kits19/d...,/content/drive/MyDrive/Seg3D/KiTS2019/kits19/d...,1
208,case_00208,/content/drive/MyDrive/Seg3D/KiTS2019/kits19/d...,/content/drive/MyDrive/Seg3D/KiTS2019/kits19/d...,1


In [8]:
import random
idc = np.random.randint(len(data))
data.iloc[0]['seg_path']

'/content/drive/MyDrive/Seg3D/KiTS2019/kits19/data/case_00000/segmentation.nii.gz'

In [11]:
import nibabel as nib 
import numpy as np 
path = data.iloc[0, 1]
img = nib.load(path)
img_arr = nib.load(path).get_fdata()
print(np.diag(abs(img.affine[[2, 1, 0, 3]])))
print(img)


[0.5        0.91992188 0.91992188 1.        ]
<class 'nibabel.nifti1.Nifti1Image'>
data shape (611, 512, 512)
affine: 
[[ 0.          0.         -0.91992188  0.        ]
 [ 0.         -0.91992188  0.          0.        ]
 [-0.5         0.          0.          0.        ]
 [ 0.          0.          0.          1.        ]]
metadata:
<class 'nibabel.nifti1.Nifti1Header'> object, endian='<'
sizeof_hdr      : 348
data_type       : b''
db_name         : b''
extents         : 0
session_error   : 0
regular         : b''
dim_info        : 0
dim             : [  3 611 512 512   1   1   1   1]
intent_p1       : 0.0
intent_p2       : 0.0
intent_p3       : 0.0
intent_code     : none
datatype        : float64
bitpix          : 64
slice_start     : 0
pixdim          : [1.        0.5       0.9199219 0.9199219 1.        1.        1.
 1.       ]
vox_offset      : 0.0
scl_slope       : nan
scl_inter       : nan
slice_end       : 0
slice_code      : unknown
xyzt_units      : 0
cal_max         : 0.0
cal_m

slice_percentage

In [10]:
data = data.sort_values(by=['case_id']).reset_index(drop=True)
data = data.drop_duplicates(keep='first')
data

,case_id,img_path,seg_path,mask
0,case_00000,/content/drive/MyDrive/Seg3D/KiTS2019/kits19/d...,/content/drive/MyDrive/Seg3D/KiTS2019/kits19/d...,1
1,case_00001,/content/drive/MyDrive/Seg3D/KiTS2019/kits19/d...,/content/drive/MyDrive/Seg3D/KiTS2019/kits19/d...,1
2,case_00002,/content/drive/MyDrive/Seg3D/KiTS2019/kits19/d...,/content/drive/MyDrive/Seg3D/KiTS2019/kits19/d...,1
3,case_00003,/content/drive/MyDrive/Seg3D/KiTS2019/kits19/d...,/content/drive/MyDrive/Seg3D/KiTS2019/kits19/d...,1
4,case_00004,/content/drive/MyDrive/Seg3D/KiTS2019/kits19/d...,/content/drive/MyDrive/Seg3D/KiTS2019/kits19/d...,1
...,...,...,...,...
205,case_00205,/content/drive/MyDrive/Seg3D/KiTS2019/kits19/d...,/content/drive/MyDrive/Seg3D/KiTS2019/kits19/d...,1
206,case_00206,/content/drive/MyDrive/Seg3D/KiTS2019/kits19/d...,/content/drive/MyDrive/Seg3D/KiTS2019/kits19/d...,1
207,case_00207,/content/drive/MyDrive/Seg3D/KiTS2019/kits19/d...,/content/drive/MyDrive/Seg3D/KiTS2019/kits19/d...,1
208,case_00208,/content/drive/MyDrive/Seg3D/KiTS2019/kits19/d...,/content/drive/MyDrive/Seg3D/KiTS2019/kits19/d...,1


In [15]:
def single_slice_percent(idx):
  row = data.iloc[idx] 
  seg_path = row['seg_path']
  seg = nib.load(seg_path)
  seg_array = seg.get_fdata()

  kidney_slice = 0
  tumor_slice  = 0
  organ_slice  = 0

  for slice_ in seg_array:
    if 1 in slice_ or 2 in slice_: organ_slice += 1
    if 1 in slice_: kidney_slice += 1 
    if 2 in slice_: tumor_slice  += 1 

  return seg_array.shape[0], organ_slice, kidney_slice, tumor_slice

In [19]:
from tqdm import tqdm 
# result = multiprocess(single_slice_percent, range(len(data)))
result = list(map(single_slice_percent, tqdm(range(len(data)))))


total_slice        = sum([result[i][0] for i in range(len(result))])
total_organ_slice  = sum([result[i][1] for i in range(len(result))])
total_organ_kidney = sum([result[i][2] for i in range(len(result))])
total_organ_tumor  = sum([result[i][3] for i in range(len(result))])

print('\nThe overall proportion of slices containing the organ in the training set:', total_organ_slice / total_slice)
print('\nThe overall proportion of slices containing the kidney in the training set:', total_organ_kidney / total_slice)
print('\nThe overall proportion of slices containing the tumor in the training set:', total_organ_tumor / total_slice)


100%|██████████| 210/210 [13:13<00:00,  3.78s/it]

The overall proportion of slices containing the organ in the training set: 0.35963367382881295
The overall proportion of slices containing the kidney in the training set: 0.34851620288834095
The overall proportion of slices containing the tumor in the training set: 0.12574850299401197


voxel_percentage


In [23]:
def single_voxel_percent(idx):
    row = data.iloc[idx]
    
    seg_path = row['seg_path']
    seg = nib.load(seg_path)
    seg_array = seg.get_fdata()

    kidney_slice = 0
    tumor_slice  = 0
    organ_slice  = 0

    for slice_ in seg_array:
      if 1 in slice_ or 2 in slice_: 
        organ_slice += 1

    organ_point  = (seg_array  > 0).astype(int).sum()
    kidney_point = (seg_array == 1).astype(int).sum()
    tumor_point  = (seg_array == 2).astype(int).sum()

    organ_slice  = organ_slice * 256 * 256

    return organ_slice, organ_point, kidney_point, tumor_point

In [24]:
# result = multiprocess(single_voxel_percent, range(len(data)), data=data)
result = list(map(single_voxel_percent, tqdm(range(len(data)))))

total_point        = sum([result[i][0] for i in range(len(result))])
total_organ_point  = sum([result[i][1] for i in range(len(result))])
total_kidney_point = sum([result[i][2] for i in range(len(result))])
total_tumor_point  = sum([result[i][3] for i in range(len(result))])

print('The overall proportion of voxels containing the organ in the training set:', total_organ_point / total_point)
print('The overall proportion of voxels containing the kidney in the training set:', total_kidney_point / total_point)
print('The overall proportion of voxels containing the tumor in the training set:', total_tumor_point / total_point)

100%|██████████| 210/210 [09:59<00:00,  2.86s/it]

The overall proportion of voxels containing the organ in the training set: 0.09507300968151484
The overall proportion of voxels containing the kidney in the training set: 0.07632667380845745
The overall proportion of voxels containing the tumor in the training set: 0.018746335873057397


Get_threshold

In [28]:
from segment.utils.file_utils import logger, load_json_file, read_yaml_file

config = read_yaml_file("/content/drive/MyDrive/Seg3D/KiTS2019/kits19/configs/preprocess_pipeline.yaml")["preprocess"]


In [27]:
def get_threshold(idx):
    row = data.iloc[idx]
    ct_path = row['img_path']
    ct      = nib.load(ct_path)
    ct_array = ct.get_fdata()

    seg_path = row['seg_path']
    seg      = nib.load(seg_path)
    seg_array = seg.get_fdata()

    organ_roi = ct_array[seg_array > 0]
    inliers = ((organ_roi < config["upper_bound"]) * (organ_roi > config["lower_bound"])).astype(int).sum()

    organ_roi_shape = organ_roi.shape[0]

    return organ_roi_shape, inliers


In [30]:
# result = multiprocess(get_threshold, range(len(data)), data=data)
result = list(map(get_threshold, tqdm(range(len(data)))))


num_point        = sum([result[i][0] for i in range(len(result))])
num_inlier       = sum([result[i][1] for i in range(len(result))])
print("Ngưỡng threshold bound = [{}, {}]  đối với kidney trong tập huấn luyện".format(config["lower_bound"], config["upper_bound"]))
print(num_inlier / num_point)

100%|██████████| 210/210 [15:16<00:00,  4.36s/it]

Ngưỡng threshold bound = [-79, 304]  đối với kidney trong tập huấn luyện
0.9899704762380368
